In [5]:
# Only if using Jupyter + PySpark
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.apache.hadoop:hadoop-aws:3.3.6,com.amazonaws:aws-java-sdk-bundle:1.12.374 pyspark-shell"


In [8]:
import findspark
findspark.init("/opt/spark")

In [9]:
from pyspark.sql import SparkSession

# Stop any old session first
try:
    spark.stop()
except:
    pass

spark = (
    SparkSession.builder
    .appName("MinioTest")
    .config("spark.master", "spark://spark-master:7077")

    # S3A core config
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minio")
    .config("spark.hadoop.fs.s3a.secret.key", "minio123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")  # <-- important for HTTP MinIO

    # Credentials providers (no AWS v2 class)
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider,"
        "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider"
    )

    # Multipart cleanup: override "24h" etc with numeric ms
    .config("spark.hadoop.fs.s3a.multipart.purge.age", "86400000")      # 24h
    .config("spark.hadoop.fs.s3a.multipart.purge.interval", "3600000")  # 1h

    # --- S3A timeouts (ms) ---
    .config("spark.hadoop.fs.s3a.connection.timeout", "60000")
    .config("spark.hadoop.fs.s3a.connection.request.timeout", "60000")
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", "60000")
    .config("spark.hadoop.fs.s3a.connection.acquisition.timeout", "60000")
    .config("spark.hadoop.fs.s3a.connection.idle.time", "60000")
    .config("spark.hadoop.fs.s3a.connection.ttl", "300000")

    # --- S3A thread pool / connection pool (ALL numeric) ---
    .config("spark.hadoop.fs.s3a.threads.max", "96")
    .config("spark.hadoop.fs.s3a.threads.keepalivetime", "60000")  # <--- key that defaulted to "60s"
    .config("spark.hadoop.fs.s3a.connection.maximum", "200")
    .config("spark.hadoop.fs.s3a.max.total.tasks", "1000")
    .config("spark.hadoop.fs.s3a.max.metadata.tasks", "100")
    .config("spark.hadoop.fs.s3a.max.total.connections", "200")

    # --- Spark-level timeouts as numbers ---
    .config("spark.files.fetchTimeout", "60000")
    .config("spark.network.timeout", "600000")
    .config("spark.rpc.askTimeout", "600000")

    # (plus the timeouts / thread configs you already set)
    .getOrCreate()
)





In [10]:
df = spark.read.json("s3a://news-raw/*.json")
df.show(5, truncate=False)


25/12/11 15:28:16 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://news-raw/*.json.
java.io.FileNotFoundException: No such file or directory: s3a://news-raw/*.json
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3866)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3688)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$getFileStatus$24(S3AFileSystem.java:3556)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.invokeTrackingDuration(IOStatisticsBinding.java:547)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.lambda$trackDurationOfOperation$5(IOStatisticsBinding.java:528)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.trackDuration(IOStatisticsBinding.java:449)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.trackDurationAndSpan(S3AFileSystem.java:2337)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.trackDurationAndSpan(S3AFileSyste

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------